<h1> HW 2: Radar Imaging Part 3a</h1>


<h3> 3. 3D Imaging using 2D Antenna Array </h3>

This task includes two parts. Our goal is to implement radar signal processing for 3D imaging
using a 2D antenna array. You will implement two algorithms we taught in the class. 

<h5> 3.1 Algorithm 1 - Conventional Beam Forming </h5>

In this part, we will need to image the scene using Algorithm 1 we taught in class for a 2D antenna array. You will generate the 3D radar heatmap by estimating the
reflected signal power from every azimuth angles $\phi$ - elevation angles $\theta$ pair. Every azimuth
angle and elevation angle, along with the range will pinpoint a 3D voxel in the spherical
coordinates. To speed up the alogrithm, you can crop the range bins to be from 100 to 120 instead of using all 512 range bins.
The code for this task should be written in the functions *beamform_2d* defined below. We provide you
the code to load the radar data in the right format and to plot the 3D heatmap in this
file.

As results, you need to include the following in your report.
1. Generate a 3D heatmap for azimuth angle $\phi$ between 60 and 130 degrees with a resolution
of 1 degree, and elevation angle $\theta$ between 70 and 110 degrees with a resolution of 1 degrees. The
definition for $\phi$ and $\theta$ can be found in Fig. 1. Include the top view, side view, and
front view projections of the 3D heatmap in the spherical coordinates, as well as a
3D point cloud by setting a threshold on the heatmap and selecting the points with a
reflection power above this threshold. Examples of this plotting code is shown at the bottom of the file.

2. Generate the above 3D heatmaps and point clouds for all data provided. 

Write your algorithm below.

In [ ]:
################# Change the values based on how much of the azimuth angles you want to see and the resolution ##################
# Define field of view in degrees that you want to process in theta, phi and range bins
def beamform_2d(beat_freq_data, phi_s, phi_e, phi_res, theta_s, theta_e, theta_res, x_idx, z_idx, r_idxs, radar_params):
    """
    Performs 2D beamforming along the azimuth (horizontal) dimension, this results in a bird eye view image.
    - beat_freq_data: beat data AKA the range FFT (size: num_x_stps * num_z_stps * num TX * num RX, num ADC samples)
    - phi_s: first azimuth angle that you want to start computing 
    - phi_e: last azimuth angle that you want to compute 
    - phi_res: resolution of the azimuth angles you want to compute
    - theta_s: first elevation angle that you want to start computing 
    - theta_e: last elevation angle that you want to compute 
    - theta_res: resolution of the elevation angles you want to compute
    - x_locs: x coordinate of antenna locations
    - z_locs: z coordinate of antenna locations
    - r_idx: range bins to calculate 
    - radar_parms: radar_params if needed 

    Returns:
    - sph_pwr: beamformed result (size: n_phi, n_theta, n_range)
    - phi: array of azimuth angles 
    - theta: array of elevation angles 
    """
    sph_pwr = 0
    theta = 0
    phi = 0 
    return sph_pwr, phi, theta 

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import os
import scipy
import scipy.io as sio
import time
import open3d as o3d
import utils 

Load data.

In [ ]:
# TODO: Put the *path* to the project folder
data_path = r"/Users/shanbhag/Documents/School/comm-proj-radars/given_data/data_003.mat"

# loading data that is given
"""
    raw_data: is the raw radar data (after mixing) of size (num_x_stp x num_rx, num_z_stp, adc_samples)
    radar_params: is a dictionary with radar and position parameters: 'sample_rate', 'num_samples', 'slope', 'lm'(lambda), 'num_x_stp', 'num_z_stp', 'num_tx', 'num_rx', 'adc_samples' 
"""
radar_params, raw_data = utils.load_raw_data(data_path)

Defining antenna positions. 

In [ ]:
x_ant_pos, z_pos, x_ant = utils.get_ant_pos_2d(radar_params['num_x_stp'], radar_params['num_z_stp'], radar_params['num_rx']) # this returns x position of rx, z positions of rx and tx, and x position of tx

Process raw data for BF algorthim (use all samples now).

In [ ]:
# TODO: reshape raw_data and process beat frequency
beat_freq = 0 

Define the angles and range bins to run, and run the BF algo.

In [ ]:
# define the azimuth angles (horizontal FOV) that we want to look at 
r_idxs = np.arange(100, 140, 1) # TODO: Change to distance rather than range bins 
phi_s, phi_e = 60, 130 
phi_res = 1
theta_s, theta_e = 70,110 
theta_res = 1

# Run your algorithm here
# TODO: input arguments that you choose
bf_output1, phi, theta= beamform_2d()

Here you should plot the outputs from beamforming.

In [ ]:
# Plot the output from 2D Beamforming (You can change this as you see fit)
fig = plt.figure(figsize=(20, 7))

to_plot = np.sum(abs(bf_output1),axis=-1)
to_plot = to_plot/np.max(np.reshape(to_plot,(1,-1))) 
to_plot = to_plot**2
ax0 = fig.add_subplot(131)
utils.plot_2d_polar_heatmap(ax0, to_plot, phi, theta, vmin=0, vmax=0.1)
ax0.title.set_text('Front View')

to_plot = np.sum(abs(bf_output1),axis=1)
to_plot = to_plot/np.max(np.reshape(to_plot,(1,-1))) 
to_plot = to_plot**2
ax1 = fig.add_subplot(132,projection = 'polar')
utils.plot_2d_heatmap(ax1, to_plot, phi, r_idxs, vmin=0, vmax=0.1)
ax1.title.set_text('Bird Eye View (Top View)')

to_plot = np.sum(abs(bf_output1),axis=0)
to_plot = to_plot/np.max(np.reshape(to_plot,(1,-1))) 
to_plot = to_plot**2
ax2 = fig.add_subplot(133)
utils.plot_2d_heatmap(ax1, to_plot, theta, r_idxs, vmin=0, vmax=0.1)
ax2.title.set_text('Side View')
plt.tight_layout()
plt.show()

<h1> HW 2: Radar Imaging Part 3b</h1>


<h5> 3.b Algorithm 3 - Matched Filter </h5>

In this part, we will need to image the scene using Algorithm 3 we taught in class for a 2D antenna array. You will generate the 3D radar heatmap by estimating
the reflected signal power from every voxel in the 3D cartesian coordinates (x,y,z).
The code for this task should be written in the function matched_filter_2d. We provide you
the code to load the radar data in the right format and to plot the heatmap in this
file. Though the MF output can be 3D, because of compute time, we ask that you just show a single depth slice of the scene making the output 2D.

As results, you need to include the following in your report: 
1. Generate the heatmap for all 3 scenes given
in the cartesian coordinates. The space of interest is indicated in Data Details in the google drive. The spatial resolution
should be 0.01 for x and z. Plot the front view.

2. Compare the heatmaps with the corresponding one from Task 3a, and comment on
the differences.

Notes: 
- Running matched filter code in Python can be slow. 
- And remember to check parameter clim to get reasonable heatmaps. 


Implement your matched filter algortihm below. If you would like to speed up processing time you can try to optimize code --> (eg. Numba, multiprocessing,), additionlly for debugging we recommend processing lower resolution.

Additionally, you may move away from a Jupyter Notebook for this task only if you find that optmizing the code requires this. However, please attach the plots in a PDF in addition to plotting the output in your code if you follow this path.

In [ ]:
######## 2D Imaging in Cartesian #############
def matched_filter_2d(raw_data_2d, num_x_cells, num_z_cells, num_y_cells, x_radar_tx, x_radar_rx, z_radar_tx, z_radar_rx):
    """
    Performs 2D matched filter on the raw data.
    - raw_data_2d: raw data  
    - num_x_cells: x axis values (azimuth)
    - num_z_cells: z axis values (elevation)
    - num_y_cells: y axis values (range)
    - x_radar_tx: transmitter positions (x) 
    - x_radar_rx: receiver positions (x)
    - z_radar_tx: transmitter positions (z)
    - z_radar_rx: receiver positions (z)
    
    Returns:
    - MF_output: matched filter output (size: num_x_cells x num_y_cells  x num_z_cells aka azimuth, depth, elevation)
    """
    MF_output = 0
    return MF_output 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy
import scipy.io as sio
import time
import open3d as o3d
import helpers 
import utils

Load data.

In [ ]:
# TODO: Put the *path* to the project folder
data_path = r"/Users/shanbhag/Documents/School/comm-proj-radars/given_data/data_003.mat"

# loading data that is given
"""
    raw_data: is the raw radar data (after mixing) of size (num_x_stp x num_rx, num_z_stp, adc_samples)
    radar_params: is a dictionary with radar and position parameters: 'sample_rate', 'num_samples', 'slope', 'lm'(lambda), 'num_x_stp', 'num_z_stp', 'num_tx', 'num_rx', 'adc_samples' 
"""
radar_params, raw_data = utils.load_raw_data(data_path)

Defining antenna positions.

In [ ]:
x_ant_pos, z_ant_pos, x_ant = utils.get_ant_pos_2d(radar_params['num_x_stp'], radar_params['num_z_stp'], radar_params['num_rx']) # this returns x position of rx, z positions of rx and tx, and x position of tx

Process raw data for MF algorthim (use all samples now).

In [ ]:
# TODO: reshape raw_data for MF function
X = 0

Define the x,y,z voxels to calculate and the radars transmitter and receiver positions in x and z. Then run your algorithm. We recommend processing at lower resolution (eg. 0.05) or smaller portions of space for debugging.

In [ ]:
# TODO: fill in the missing values below
num_x_cells = 0 
num_z_cells = 0
num_y_cells = 0

# transmitters x position is one every lambda (repeat by four to match size of x_radar_rx) and shift by the physical offset on the board
x_radar_tx = 0 
x_radar_rx = 0  
z_radar_tx = 0  
z_radar_rx = 0  

# Run matched filter algorithm
MF_output = matched_filter_2d()

Plot your outputs below.

In [ ]:
# Plot the output from MF (You can change this as you see fit (this will just plot the front of the object as we are not processing the entire heatmap)
fig = plt.figure(figsize=(7, 7))

to_plot = np.sum(abs(MF_output),axis=-1)
to_plot = to_plot/np.max(np.reshape(to_plot,(1,-1))) 
to_plot = to_plot**2
ax0 = fig.add_subplot(111)
utils.plot_2d_heatmap(ax0, to_plot, num_x_cells, num_y_cells, vmin=0, vmax=0.1)
ax0.title.set_text('Bird Eye View (Top View)')

plt.tight_layout()
plt.show()
